In [1]:
import ase
import requests
import numpy as np
import json
import atoms_utils
from pymatgen import MPRester, Structure
from pymatgen.io.ase import AseAtomsAdaptor
from cStringIO import StringIO
from os import remove

/usr/local/miniconda3/envs/aiida/lib/python2.7/site-packages/pymatgen/__init__.py:87: UserWarning: 
Pymatgen will drop Py2k support from v2019.1.1. Pls consult the documentation
at https://www.pymatgen.org for more details.
  at https://www.pymatgen.org for more details.""")


In [2]:
def anonymize(structure):
    n_atoms = len(structure.positions)
    structure.set_atomic_numbers([1] * n_atoms)
    structure.set_chemical_symbols(['H'] * n_atoms)
    return structure

def scale(structure, per='site'):
    if per == 'site':
        n_atoms = len(structure.positions)
    elif per == 'cell':
        n_atoms = 1  # scaling volume to 1/cell is the same as having 1 atom
    else:
        return ExitCode(400, '{} is not a valid `per` for scaling'.format(per))
    cell_volume = np.dot(np.cross(structure.cell[0], structure.cell[1]), structure.cell[2])
    new_cell = structure.get_cell() / np.cbrt(cell_volume / n_atoms)
    structure.set_cell(new_cell)
    new_pos = structure.get_positions() / \
        np.linalg.norm(structure.get_cell(), axis=1) * \
        np.linalg.norm(new_cell, axis=1)
    structure.set_positions(new_pos)
    return structure

In [3]:
structures = {}
with MPRester('0WqdPfXxloze6T9N') as mpr:
    structures['diamond'] = mpr.get_structure_by_material_id("mp-66")
    structures['gaas'] = mpr.get_structure_by_material_id("mp-2534")
    structures['rocksalt'] = mpr.get_structure_by_material_id("mp-22862")
    structures['perovskite'] = mpr.get_structure_by_material_id("mp-5827")
    structures['zincblende'] = mpr.get_structure_by_material_id("mp-10695")
    structures['wurtzite'] = mpr.get_structure_by_material_id("mp-561258")
    structures['fcc'] = mpr.get_structure_by_material_id("mp-30")
    structures['big_fcc'] = mpr.get_structure_by_material_id("mp-12628")
    structures['bcc'] = mpr.get_structure_by_material_id("mp-13")
    structures['triclinic'] = mpr.get_structure_by_material_id("mp-9481")
atoms = {name: AseAtomsAdaptor.get_atoms(structure) for name, structure in structures.iteritems()}
spkitMax = {1: max([len(structure.sites) for structure in structures.values()])}

/usr/local/miniconda3/envs/aiida/lib/python2.7/site-packages/pymatgen/ext/matproj.py:111: UserWarning: If you query for structure data encoded using MP's Structure Notation Language (SNL) format and you use `mp_decode=True` (the default) for MPRester queries, you should install dependencies via `pip install pymatgen[matproj.snl]`.
  warnings.warn("If you query for structure data encoded using MP's "


In [4]:
atoms = {name: scale(anonymize(atom)) for name, atom in atoms.iteritems()}

In [5]:
atom_dicts = {name: atoms_utils.as_dict(atom) for name, atom in atoms.iteritems()}

In [12]:
payload = {'atoms': json.dumps(atom_dicts.values()), 'nocenters': None,
           'gaussian_width': 0.5, 'cutoff': 3.5,
           'cutoff_transition_width': 0.5, 'nmax': 8,
           'lmax': 6}
# r = requests.get('http://127.0.0.1:8899/v1/get_Soaps/', params=payload)
r = requests.get('http://127.0.0.1:5000/v1/fingerprints/soaps/', params=payload)
results = r.json()

In [6]:
soaps = {}
for name, atom_dict in atom_dicts.iteritems():
    payload = {'atoms': json.dumps(atom_dict),
               'spkitMax': json.dumps(spkitMax), 'nocenters': None,
               'gaussian_width': 0.5, 'cutoff': 3.5,
               'cutoff_transition_width': 0.5, 'nmax': 8,
               'lmax': 6}
    # r = requests.get('http://127.0.0.1:8899/v1/get_soap/', params=payload)
    # r = requests.get('http://127.0.0.1:5000/v1/fingerprints/soap/', params=payload)
    soaps[name] = r.json()